In [19]:
# Example that shows all active positions in the 0.3% USDC/ETH
# pool using data from the Uniswap v3 subgraph.

from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import math

# 0.3% USDC/ETH pool
# pool id 是池子的合约地址
POOL_ID = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"

# If you want to test with a smaller pool:
# 0.05% GUSD/DAI pool (small pool)
#POOL_ID = "0x7cf12cef5ce9e5e068ebdef470ff8295e26c47b9"

URL = "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3"

# 获取一个池子的信息
pool_query = """query get_pools($pool_id: ID!) {
  pools(where: {id: $pool_id}) {
    tick
    sqrtPrice
    liquidity
    feeTier
    token0 {
      symbol
      decimals
    }
    token1 {
      symbol
      decimals
    }
  }
}"""

# return open positions only (with liquidity > 0)
# 获取池子里的一个头寸
position_query = """query get_positions($pool_id: ID!) {
  positions(skip: $num_skip, where: {pool: $pool_id, liquidity_gt: 0}) {
    id
    tickLower { tickIdx }
    tickUpper { tickIdx }
    liquidity
  }
}"""

block_query = """query {
  _meta {
    block {
      number
    }
  }
}"""
     

In [20]:

TICK_BASE = 1.0001

# tick 和价格的转换方法：
#  p = 1.0001 ** i
def tick_to_price(tick):
    return TICK_BASE ** tick

In [21]:

client = Client(
    transport=RequestsHTTPTransport(
        url=URL,
        verify=True,
        retries=5,
    ))
     

In [ ]:
# get position info
# 读取池子里的头寸 放在positions列表里

positions = []
num_skip = 0
try:
    while True:
        print("Querying positions, num_skip={}".format(num_skip))
        variables = {"num_skip": num_skip, "pool_id": POOL_ID}
        response = client.execute(gql(position_query), variable_values=variables)

        if len(response["positions"]) == 0:
            break
        num_skip += len(response["positions"])
        for item in response["positions"]:
            tick_lower = int(item["tickLower"]["tickIdx"])
            tick_upper = int(item["tickUpper"]["tickIdx"])
            liquidity = int(item["liquidity"])
            id = int(item["id"])
            positions.append((tick_lower, tick_upper, liquidity, id))
except Exception as ex:
    print("got exception while querying position data:", ex)
    exit(-1)

print("Total number of positions found = {}".format(len(positions)))

In [26]:
len(positions)

[(191820, 205680, 703896364746, 100209),
 (138180, 207240, 395460364753, 10023),
 (191820, 205680, 3519373165432, 100258),
 (194400, 194880, 16338467085711, 10038),
 (193200, 200340, 9334307987607, 100501),
 (197280, 199380, 15160750406009, 100640),
 (189840, 208200, 4737114837605, 100684),
 (191340, 205200, 962663212755, 100859),
 (191280, 205140, 600575337770, 100930),
 (189300, 205200, 2421955617446, 101074),
 (197640, 198960, 87822561714309, 101262),
 (191160, 193380, 601997327259, 10137),
 (184200, 276300, 34235428827, 10141),
 (194520, 195300, 15474398912031, 10148),
 (197280, 197880, 1161227933841289, 101736),
 (195420, 203460, 10027018335017, 101923),
 (191160, 193380, 12039946545193, 10193),
 (194760, 202380, 1196654696052, 101971),
 (161160, 214200, 19602666114797, 10201),
 (190800, 204660, 1372434178929, 102016),
 (194520, 194580, 8313202572502, 10207),
 (121860, 276300, 51853265344561, 102091),
 (190500, 204420, 13220441228968, 102242),
 (186780, 414480, 11991237557306, 102

In [7]:

try:
    variables = {"pool_id": POOL_ID}
    response = client.execute(gql(pool_query), variable_values=variables)

    if len(response['pools']) == 0:
        print("pool not found")
        exit(-1)

    pool = response['pools'][0]
    pool_liquidity = int(pool["liquidity"])
    current_tick = int(pool["tick"])

    token0 = pool["token0"]["symbol"]
    token1 = pool["token1"]["symbol"]
    decimals0 = int(pool["token0"]["decimals"])
    # decimal 表示 amount = amount * 10 ** decimals
    decimals1 = int(pool["token1"]["decimals"])
except Exception as ex:
    print("got exception while querying pool data:", ex)
    exit(-1)

In [10]:
response

{'pools': [{'tick': '205606',
   'sqrtPrice': '2308575120740626481872688152876237',
   'liquidity': '12865925662732103842',
   'feeTier': '3000',
   'token0': {'symbol': 'USDC', 'decimals': '6'},
   'token1': {'symbol': 'WETH', 'decimals': '18'}}]}

In [24]:

# get block height, for record
try:
    response = client.execute(gql(block_query))
    blocknum = response['_meta']['block']['number']
    print("Ethereum block height {}".format(blocknum))
except Exception as ex:
    print("got exception", ex)


Ethereum block height 16202890


In [ ]:
# Compute and print the current price
# 
current_price = tick_to_price(current_tick)
# 价格的根号
current_sqrt_price = tick_to_price(current_tick / 2)
# 当前的价格等于池子内y的数量除x的数量（加上decimal的调整）
adjusted_current_price = current_price / (10 ** (decimals1 - decimals0))
print("Current price={:.6g} {} for {} at tick {}".format(adjusted_current_price, token1, token0, current_tick))

# Sum up all the active liquidity and total amounts in the pool
active_positions_liquidity = 0
total_amount0 = 0
total_amount1 = 0

# Print all active positions
# 遍历池子里的每一个tick范围，计算每一个tick范围内的头寸的数量，然后加总到total amount中
for tick_lower, tick_upper, liquidity, id in sorted(positions):

    sa = tick_to_price(tick_lower / 2)
    sb = tick_to_price(tick_upper / 2)

    # 当前的价格超过了tick范围的上限的时候，你的流动性中只有token1存在了（因为人们都来用token0来换token1）
    if tick_upper < current_tick:
        # Only token1 locked
        # 这个公式推导在formulas/calculate-v3-liquidity.PNG里有
        amount1 = liquidity * (sb - sa) # eq(9) in technical note
        total_amount1 += amount1

    # 当前的价格在tick的范围之内，你的流动性头寸中两种币都存在，需要分别计算
    elif tick_lower <= current_tick <= tick_upper:
        # Both tokens present
        amount0 = liquidity * (sb - current_sqrt_price) / (current_sqrt_price * sb) # eq(12) in technical note
        amount1 = liquidity * (current_sqrt_price - sa) # eq(13) in technical note
        adjusted_amount0 = amount0 / (10 ** decimals0)
        adjusted_amount1 = amount1 / (10 ** decimals1)

        total_amount0 += amount0
        total_amount1 += amount1
        active_positions_liquidity += liquidity

        print("  position {: 7g} in range [{},{}]: {:.6g} {} and {:.6g} {} at the current price".format(
              id, tick_lower, tick_upper,
              adjusted_amount0, token0, adjusted_amount1, token1))
    # 当前的价格小于tick范围的下限，你的流动性中只有token0，因为人们都来用token0来换token1）
    else:
        # Only token0 locked
        amount0 = liquidity * (sb - sa) / (sa * sb) # eq(4) in technical note
        total_amount0 += amount0


print("In total (including inactive positions): {:.6g} {} and {:.6g} {}".format(
      total_amount0 / 10 ** decimals0, token0, total_amount1 / 10 ** decimals1, token1))
print("Total liquidity from active positions: {}, from pool: {} (should be equal...)".format(
      active_positions_liquidity, pool_liquidity))